[참고: mastering decorators](https://levelup.gitconnected.com/mastering-decorators-in-python-3-588cb34fff5e)

파이썬은 일급 프로그래밍 언어   
함수는 오브젝터처럼 인자나 반환값으로 사용될수 있다는 의미 

In [1]:
def pick_roses(color):
    return f"getting her {color} roses!"

#함수를 인자로 전달한다.
def get_her_flowers(picker):
    return picker("pink")

get_her_flowers(pick_roses)

'getting her pink roses!'

### Function Decorators
함수 데코레이터는 함수를 입력으로 한다.

### Case 1- decorators without direct input

In [11]:
class Decorator(object):

    def __init__(self, f):
        print(f"inside __init__() with input function {f.__name__}")
        self.f = f  # function definition is complete and is available here

    def __call__(self, *args, **kwargs):
        print(f"inside __call__() with args: {args} and kwargs: {kwargs}")
        self.f(*args, **kwargs)


@Decorator
def my_funcion(a, b, c):
    print("inside my_funcion()")

@Decorator
def my_funcion_no_call(a, b, c):
    print("inside my_funcion_no_call()")
    
print("finished decorating my_funcion()")
my_funcion(1, 2, 3)
print("immediately after my_function() line")

inside __init__() with input function my_funcion
inside __init__() with input function my_funcion_no_call
finished decorating my_funcion()
inside __call__() with args: (1, 2, 3) and kwargs: {}
inside my_funcion()
immediately after my_function() line


* Decoration Time

In [12]:
def my_function(a,b,c):
    print("inside my_function()")
    
my_function = Decorator(my_function)

inside __init__() with input function my_function


In [14]:
type(my_function)

__main__.Decorator

* Invocation time

In [4]:
 my_function(a=1,b=2,c=3)

inside __call__() with args: () and kwargs: {'a': 1, 'b': 2, 'c': 3}
inside my_function()


### Case2 - decorators with direct input
데코레이터에 인자를 전달하고 싶을 때   
__init__ 함수에 인자를 전달하면 된다. 

In [16]:
class Decorator(object):

    def __init__(self, *decorator_args, **decorator_kwargs):
        print(f"inside __init__() with input function with args {decorator_args} and kwargs {decorator_kwargs}")

    def __call__(self, f):
        print(f"inside __call__() with function {f.__name__}")

        def wrapped(*args, **kwargs):
            print(f"inside wrapped with args {args} and kwargs {kwargs}")
            return f(*args, **kwargs)

        return wrapped


@Decorator("decorator arg 1", "decorator arg 2")
def my_funcion(a, b, c):
    print("inside my_funcion()")


@Decorator("decorator arg 3", "decorator arg 4")
def my_funcion_no_call(a, b, c):
    print("inside my_funcion_no_call()")

print("finished decorating my_funcion()")
my_funcion(1, 2, 3)
print("immediately after my_function() line")

inside __init__() with input function with args ('decorator arg 1', 'decorator arg 2') and kwargs {}
inside __call__() with function my_funcion
inside __init__() with input function with args ('decorator arg 3', 'decorator arg 4') and kwargs {}
inside __call__() with function my_funcion_no_call
finished decorating my_funcion()
inside wrapped with args (1, 2, 3) and kwargs {}
inside my_funcion()
immediately after my_function() line


* Decoration Time

In [17]:
def my_function(a, b, c):
    print("inside my_funcion()")

my_function = Decorator("arg1","arg2")(my_function)

inside __init__() with input function with args ('arg1', 'arg2') and kwargs {}
inside __call__() with function my_function


In [18]:
type(my_function)

function

* Invocation Time

In [9]:
my_function(1,2,3)

inside wrapped with args (1, 2, 3) and kwargs {}
inside my_funcion()


### Class Decorators
클래스 데코레이터는 파이썬 클래스 객체를 데코레이트한다.   
함수와 똑같은 작용을 한다. 데코리이트 하는 객체는 클래스인것만 빼고 말이다.

In [5]:
def time_this(func):
    def wrapped(*args, **kwargs):
        print("_________timer starts_________")
        from datetime import datetime
        before = datetime.now()
        x = func(*args, **kwargs)
        after = datetime.now()
        print("** elapsed Time = {0} **\n".format(after - before))
        return x

    return wrapped


def time_all_class_methods(Cls):
    # decoration body - doing nothing really since we need to wait until the decorated object is instantiated
    class Wrapper:
        def __init__(self, *args, **kwargs):
            print(f"__init__() called with args: {args} and kwargs: {kwargs}")
            self.decorated_obj = Cls(*args, **kwargs)

        def __getattribute__(self, s):
            try:
                x = super().__getattribute__(s)
                return x
            except AttributeError:
                pass
            x = self.decorated_obj.__getattribute__(s)
            if type(x) == type(self.__init__):  # it is an instance method
                print(f"attribute belonging to decorated_obj: {s}")
                return time_this(x)  # this is equivalent of just decorating the method with time_this
            else:
                return x

    return Wrapper  # decoration ends here


@time_all_class_methods
class MyClass:
    def __init__(self):
        import time
        print('entering MyClass.__init__')
        time.sleep(1.8)
        print("exiting MyClass.__init__")

    def method_x(self):
        print("entering MyClass.method_x")
        import time
        time.sleep(0.7)
        print("exiting MyClass.method_x")

    def method_y(self):
        print("entering MyClass.method_x")
        import time
        time.sleep(1.2)
        print("exiting MyClass.method_x")

In [6]:
instance = MyClass()

__init__() called with args: () and kwargs: {}
entering MyClass.__init__
exiting MyClass.__init__


In [7]:
instance.method_x()
instance.method_y()

attribute belonging to decorated_obj: method_x
_________timer starts_________
entering MyClass.method_x
exiting MyClass.method_x
** elapsed Time = 0:00:00.703178 **

attribute belonging to decorated_obj: method_y
_________timer starts_________
entering MyClass.method_x
exiting MyClass.method_x
** elapsed Time = 0:00:01.201058 **



### __get_attribute__

In [36]:
class A:
    def __getattribute__(self,name):
        print("getattribute 호출")
        return ('aaa-'+name)
    
a = A()
a.ace = "test"
print(a.ace)

getattribute 호출
aaa-ace
